## 准备工作
### 安装PaddleSeg以及更新依赖
> 这里使用的PaddleSeg的2.3版本

In [ ]:
# 解压PaddleSeg套件
!unzip -oq /home/aistudio/data/data114849/PaddleSeg-release-2.3.zip

In [ ]:
# 为方便后期操作将文件夹改名
!mv PaddleSeg-release-2.3 PaddleSeg

In [ ]:
# 安装依赖，AIStudio实验环境已有相关依赖
#!pip install -r PaddleSeg/requirements.txt

### 解压数据集

In [2]:
!unzip -oq /home/aistudio/data/data77571/train_and_label.zip -d data/
!unzip -oq /home/aistudio/data/data77571/img_test.zip -d data/

### 生成训练列表文件
> 这里直接按比例分割，没有用shuffle

In [4]:
import os
img_train_dir = os.listdir('./data/img_train')
train_val_ratio = 0.9 #划分训练集和验证集的比例
train_img_num = int(len(img_train_dir)*train_val_ratio)
val_img_num = len(img_train_dir) - train_img_num
train_list_origin = img_train_dir[:train_img_num]
val_list_origin = img_train_dir[train_img_num:]
print(len(train_list_origin),len(val_list_origin)) #输出训练集和验证集数量

#生成trainlist
with open('train_list.txt','w') as f:
    for i in range(len(train_list_origin)):
        train_img_name = 'img_train/' + train_list_origin[i]
        train_lab_name = 'lab_train/' + train_list_origin[i].split('.')[0] + '.png'
        f.write(train_img_name + ' ' + train_lab_name + '\n')
#生成vallist
with open('val_list.txt','w') as f:
    for i in range(len(val_list_origin)):
        val_img_name = 'img_train/' + val_list_origin[i]
        val_lab_name = 'lab_train/' + val_list_origin[i].split('.')[0] + '.png'
        f.write(val_img_name + ' ' + val_lab_name + '\n')

59986 6666


## 模型选择和实践

### Swin Transformer简介
> Swin Transformer是ViT的一大进步，建议参考李沐大神读论文的方法啃原论文

[Swin Transformer: Hierarchical Vision Transformer using Shifted Windows](https://arxiv.org/pdf/2103.14030.pdf)

![](https://ai-studio-static-online.cdn.bcebos.com/a36bca046e8e45a597946f2463611c1aad4891a6b894481e8675688435c23408)



### 模型训练

In [ ]:
!python PaddleSeg/train.py \
       --config swin.yml \
       --do_eval \
       --use_vdl \
       --save_interval 4000 \
       --save_dir output

### 模型预测

In [ ]:
!python PaddleSeg/predict.py \
       --config swin0.yml \
       --model_path output/best_model/model.pdparams \
       --image_path data/img_testA \
       --save_dir ./result

> 前往output\home\aistudio\result\pseudo_color_prediction打包预测结果

## 改进想法
1. 面对数据集不均衡现象采用欠采样或过采样方法重新处理数据集
1. 对欠采样的数据集使用不同的数据增强方法
1. Loss使用LovaszSoftmaxLoss

## 参考链接

* [2020 CCF BDCI 地块分割Top1方案 & 语义分割trick整理](https://zhuanlan.zhihu.com/p/346862877)

* [飞桨常规赛：遥感影像地块分割-7月第3名方案（增强修改版）](https://aistudio.baidu.com/aistudio/projectdetail/1789075)

* [飞桨常规赛：遥感影像地块分割8月第1名方案](https://aistudio.baidu.com/aistudio/projectdetail/2284066)